In [ ]:
import csv
import datetime
import re
import xlsxwriter

import csv
import datetime

notas = {}
notas_canceladas = []

try:
    with open('notas.csv', 'r') as archivo_csv:
        lector_csv = csv.reader(archivo_csv)
        notas.clear()
        notas_canceladas.clear()
        folio_actual = None
        detalle_nota = []
        for i, fila in enumerate(lector_csv):
            if i == 0:
                continue
            folio = int(fila[0])
            if fila[1] == 'CANCELADA':
                notas_canceladas.append(folio)
            else:
                fecha_str = fila[1]
                if fecha_str:
                    fecha = datetime.datetime.strptime(fecha_str, "%Y-%m-%d").date()
                else:
                    fecha = None
                cliente = fila[2]
                correo = fila[3]
                rfc = fila[4]
                if len(fila) >= 7 and fila[5] and fila[6]:
                    servicio = fila[5]
                    monto = float(fila[6])
                    detalle_nota.append([servicio, monto])
                else:
                    if folio_actual:
                        notas[folio_actual] = (fecha, cliente, correo, rfc, detalle_nota)
                    folio_actual = folio
                    detalle_nota = []
        if folio_actual:
            notas[folio_actual] = (fecha, cliente, correo, rfc, detalle_nota)
except FileNotFoundError:
    print('\n** No se encontró un archivo de datos CSV. Se inicia con un estado inicial vacío. **')


def validar_correo(correo):
    patron = r'^[\w\.-]+@[\w\.-]+\.\w+$'
    if re.match(patron, correo):
        return True
    else:
        return False


def validar_rfc(rfc):
    patron = r'^[A-Z]{4}[0-9]{6}[A-Z0-9]{3}$'
    if re.match(patron, rfc):
        return True
    else:
        return False


def validar_monto(monto_str):
    try:
        monto = float(monto_str)
        return monto >= 0.01
    except ValueError:
        return False


def registrar_notas():
    total = 0
    while True:
        print('\n   REGISTRA UNA NUEVA NOTA')
        detalle_nota = []
        print('\nNota: Escriba (salir) si desea volver al menú principal')
        while True:
            cliente = validacion_txt('\nIngrese el nombre del cliente: ')
            if cliente == 'salir':
                print('\n** OPERACIÓN CANCELADA, VOLVIENDO AL MENÚ PRINCIPAL **')
                return
            elif cliente:
                break
        while True:
            try:
                fecha = input('\nIngrese la fecha en formato DD/MM/YYYY: ')
                fecha_actual = datetime.date.today()
                fecha = datetime.datetime.strptime(fecha, "%d/%m/%Y").date()
                if fecha > fecha_actual:
                    print('\n** LA FECHA INGRESADA NO DEBE SER POSTERIOR A LA FECHA ACTUAL, INGRESE UNA FECHA VÁLIDA **')
                else:
                    break
            except ValueError:
                print('** EL FORMATO DE FECHA NO ES VÁLIDO, INGRESE LA FECHA EN FORMATO DD/MM/YYYY **')

        while True:
            servicio = input('\nIngrese el servicio que se desea solicitar: ')
            if not servicio:
                print('\n** EL SERVICIO NO SE PUEDE OMITIR, INGRESE UN SERVICIO **')
                continue
            while True:
                try:
                    monto = input('\nIngrese el monto a pagar: ')
                    monto = float(monto)
                    if validar_monto(monto):
                        break
                    else:
                        print('\n**  EL MONTO INGRESADO DEBE SER MAYOR A 0 **')
                except ValueError:
                    print('\n** EL MONTO INGRESADO NO ES UN NÚMERO VÁLIDO, INGRESELO NUEVAMENTE  **')
            total += monto
            detalle_nota.append([servicio, monto])
            servicio_n = input('\n¿Desea agregar otro servicio? (S)i (N)o: ')
            if servicio_n.lower() != 's':
                break

        while True:
            correo = input('\nIngrese su correo electrónico: ')
            if validar_correo(correo):
                break
            else:
                print('\n**  EL CORREO PROPORCIONADO NO TIENE UN FORMATO VÁLIDO, INGRESELO NUEVAMENTE **')
        while True:
            rfc = input('\nIngresa tu RFC : ')
            rfc = rfc.strip().upper()
            if validar_rfc(rfc):
                break
            else:
                print('**  EL RFC INGRESADO NO CUMPLE CON EL FORMATO, INGRESELO NUEVAMENTE  **')
        nueva_nota = (fecha, cliente, correo, rfc, detalle_nota)
        notas[len(notas) + 1] = nueva_nota
        agregar_nota = input('\n¿Desea registrar otra nota? (S)i (N)o: ')
        if agregar_nota.lower() != 's':
            print('\n** NOTA(S) REGISTRADA CORRECTAMENTE **')
            print(notas)
            break


def consulta_periodo():
    print('\n   CONSULTA DE NOTAS POR PERÍODO ')
    while True:
        try:
            fecha_inicial_str = input('\nIngrese la fecha inicial (DD/MM/YYYY) o presione Enter para usar fecha dada por el sistema (01/01/2000): ')
            if fecha_inicial_str == '':
                fecha_inicial = datetime.date(2000, 1, 1)
            else:
                fecha_inicial = datetime.datetime.strptime(fecha_inicial_str, "%d/%m/%Y").date()
            fecha_final_str = input('\nIngrese la fecha final (DD/MM/YYYY) o presione Enter para usar la fecha actual: ')
            if fecha_final_str == '':
                fecha_final = datetime.date.today()
            else:
                fecha_final = datetime.datetime.strptime(fecha_final_str, "%d/%m/%Y").date()
            if fecha_final < fecha_inicial:
                print('\n** LA FECHA FINAL DEBE SER MAYOR O IGUAL QUE LA FECHA ACTUAL. **')
            else:
                break
        except ValueError:
            print('\n** FORMATO DE FECHA INCORRECTO, UTILICE EL FORMATO (DD/MM/YYYY). **')
    notas_periodo = []
    for folio, (fecha, cliente, correo, rfc, detalle_nota) in notas.items():
        if fecha_inicial <= fecha <= fecha_final:
            total = sum(monto for _, monto in detalle_nota)
            notas_periodo.append((folio, fecha, cliente, correo, rfc, total))
    if not notas_periodo:
        print(f'\n** No hay notas registradas en el período de {fecha_inicial} a {fecha_final}. **')
    else:
        print('\n** NOTAS REGISTRADAS EN EL PERIODO INGRESADO: **')
        print('---------------------------------------------------')
        for folio, fecha, cliente, correo, rfc, total in notas_periodo:
            print(f'Folio: {folio}')
            print(f'Fecha: {fecha}')
            print(f'Nombre del cliente: {cliente}')
            print(f'Correo del cliente: {correo}')
            print(f'RFC del cliente: {rfc}')
            print(f'Total: ${total:.2f}')
            print('Detalle de la nota:')
            for servicio, costo in detalle_nota:
                print(f'  Servicio: {servicio}')
                print(f'  Costo: ${costo:.2f} pesos')


def consulta_folio():
    print('\n   CONSULTA NOTA POR FOLIO ')
    folio_consulta = validacion_num('\nIngrese el folio de la nota que desea buscar: ')
    if folio_consulta in notas:
        fecha, nombre, correo, rfc, detalle_nota = notas[folio_consulta]
        total = sum(monto for _, monto in detalle_nota)
        print(f'\n** NOTA CONSULTADA **')
        print('--------------------------------')
        print(f'Folio: {folio_consulta}')
        print(f'Fecha: {fecha}')
        print(f'Nombre del cliente: {nombre}')
        print(f'Correo del cliente: {correo}')
        print(f'RFC del cliente: {rfc}')
        print(f'Total: ${total:.2f} pesos')
        print('Detalle de la nota:')
        for servicio, costo in detalle_nota:
            print(f'  Servicio: {servicio}')
            print(f'  Costo: ${costo:.2f} pesos')
    else:
        print('\n** La nota no existe **')


def consulta_cliente():
    print('\n   CONSULTA NOTAS POR CLIENTE ')
    notas_por_cliente = {}
    rfcs = list(set([nota[2] for nota in notas.values()]))

    rfcs.sort()

    for idx, rfc in enumerate(rfcs, start=1):
        notas_por_cliente[idx] = rfc
        print(f'{idx}. RFC: {rfc}')
    while True:
        try:
            seleccion = validacion_num('\nSeleccione el número de RFC a consultar (0 para regresar): ')
            if seleccion == 0:
                return
            elif 1 <= seleccion <= len(rfcs):
                break
            else:
                print('\n** Selección no válida. Ingrese un número válido **')
        except ValueError:
            print('\n** Opción no válida. Ingrese un número válido **')

    rfc_seleccionado = notas_por_cliente[seleccion]
    notas_cliente = [(folio, fecha, nombre, total, detalle) for folio, (fecha, nombre, _, total, detalle) in notas.items() if
                     _ == rfc_seleccionado]
    if not notas_cliente:
        print(f'\n** No hay notas registradas para el cliente con RFC {rfc_seleccionado}. **')
    else:
        print(f'\n** NOTAS REGISTRADAS PARA EL CLIENTE CON RFC {rfc_seleccionado}: **')
        print('--------------------------------------------------------')
        for folio, fecha, nombre, total, detalle in notas_cliente:
            print(f'Folio: {folio}, Fecha: {fecha}, Cliente: {nombre}, Total: ${total:.2f}')
            print('Detalle de la nota:')
            for servicio in detalle:
                nombre_servicio, costo_servicio = servicio
                print(f'  Servicio: {nombre_servicio}')
                print(f'  Costo: ${costo_servicio:.2f}')
        monto_promedio = sum([total for _, _, _, total, _ in notas_cliente]) / len(notas_cliente)
        print(f'\nMonto promedio de las notas del cliente: ${monto_promedio:.2f}')
        exportar_excel = input('\n¿Desea exportar esta información a un archivo Excel? (S)i (N)o: ')
        if exportar_excel.lower() == 's':
            nombre_archivo = f'{rfc_seleccionado}_{datetime.datetime.now().strftime("%Y%m%d%H%M%S")}.xlsx'
            workbook = xlsxwriter.Workbook(nombre_archivo)
            worksheet = workbook.add_worksheet()
            headers = ['Folio', 'Fecha', 'Cliente', 'Total', 'Servicio', 'Costo']
            for col, header in enumerate(headers):
                worksheet.write(0, col, header)
            row = 1
            for folio, fecha, nombre, total, detalle in notas_cliente:
                for servicio in detalle:
                    nombre_servicio, costo_servicio = servicio
                    worksheet.write(row, 0, folio)
                    worksheet.write(row, 1, fecha)
                    worksheet.write(row, 2, nombre)
                    worksheet.write(row, 3, total)
                    worksheet.write(row, 4, nombre_servicio)
                    worksheet.write(row, 5, costo_servicio)
                    row += 1
            workbook.close()

            print(f'\n** Información exportada a {nombre_archivo} correctamente. **')


def cancelar_nota():
    print('\n   CANCELAR UNA NOTA')
    while True:
        folio_cancelar = input('\nIngrese el folio de la nota a cancelar o escriba "salir" si quiere regresar al menú principal: ')
        if folio_cancelar.lower() == 'salir':
            print("\nRegresando al menú principal...")
            return
        try:
            folio_cancelar = int(folio_cancelar)
            if folio_cancelar in notas and folio_cancelar not in notas_canceladas:
                fecha, nombre, total, detalle_nota = notas[folio_cancelar]
                print(f'\n** NOTA A PUNTO DE CANCELAR: **')
                print('--------------------------------')
                print(f'Folio: {folio_cancelar}')
                print(f'Fecha: {fecha}')
                print(f'Nombre del cliente: {nombre}')
                print(f'Total: ${total:.2f} pesos')
                print('Servicios:')
                for servicio in detalle_nota:
                    nombre_servicio, costo_servicio = servicio
                    print(f'  Nombre: {nombre_servicio}')
                    print(f'  Costo: ${costo_servicio:.2f} pesos')

                confirmacion = input('\n¿Desea cancelar esta nota? (S)i (N)o: ')
                if confirmacion.lower() == 's':
                    notas_canceladas.append(folio_cancelar)
                    print('\n** NOTA CANCELADA CORRECTAMENTE **')
                else:
                    print('\n** Operación cancelada **')
                    continue
            else:
                print('** La nota no existe o ya está cancelada **')
        except ValueError:
            print('** Opción no válida, ingrese un número válido.')


def recuperar_nota():
    print('\n   RECUPERAR UNA NOTA CANCELADA ')
    if not notas_canceladas:
        print('**  No hay notas canceladas para recuperar **')
        return

    print('\n** NOTAS PREVIAMENTE CANCELADAS: **')
    print('---------------------------------------')
    for folio in notas_canceladas:
        fecha, nombre, total, _ = notas[folio]
        print(f'Folio: {folio}, Fecha: {fecha}, Cliente: {nombre}, Total: ${total:.2f}')

    while True:
        folio_recuperar = input('\nIngrese el folio de la nota a recuperar (o 0 para cancelar): ')
        try:
            folio_recuperar = int(folio_recuperar)
            if folio_recuperar in notas_canceladas:
                notas_canceladas.remove(folio_recuperar)
                print(f'\n** Nota {folio_recuperar} recuperada **')
                break
            elif folio_recuperar == 0:
                print('** Operación cancelada, volviendo al menú principal **')
                break
            else:
                print('** El folio ingresado no corresponde a una nota cancelada **')
        except ValueError:
            print('** Opción no válida, ingrese un número válido **')


def validacion_txt(mensaje):
    while True:
        entrada = input(mensaje)
        if entrada.strip() == "":
            print('\n** NO SE PUEDE OMITIR EL CAMPO, INGRESE UN REGISTRO O ESCRIBA SALIR PARA VOLVER AL MENU_PRINCIPAL **')
        else:
            return entrada


def validacion_num(mensaje):
    while True:
        entrada = input(mensaje)
        if entrada.strip() == "":
            print('\n** NO SE PUEDE OMITIR LA OPERACIÓN, INGRESE ALGUNA OPCIÓN **')
            continue
        try:
            numero = int(entrada)
            return numero
        except ValueError:
            print('\n** OPCIÓN NO VÁLIDA, INGRESE UN NÚMERO DE ALGÚNA OPCIÓN MOSTRADA **')

def menu_principal():
  with open('notas.csv', 'r') as archivo_csv:
    lector_csv = csv.reader(archivo_csv)
    notas.clear()
    notas_canceladas.clear()

    while True:
        print('\n---------------------------------------------')
        print('       MENÚ PRINCIPAL TALLER MECÁNICO         ')
        print('---------------------------------------------')
        print('1. Registrar una nota')
        print('2. Consultas y reportes')
        print('3. Cancelar una nota')
        print('4. Recuperar una nota')
        print('5. Salir')
        opcion = input('\nIngrese el número de la operación que desea realizar: ')

        if opcion == '1':
            registrar_notas()
        elif opcion == '2':
            submenu_consultas()
        elif opcion == '3':
            cancelar_nota()
        elif opcion == '4':
            recuperar_nota()
        elif opcion == '5':
            confirmacion_salir = input('\n¿Está seguro de que desea salir del programa? (S)i (N)o: ')
            if confirmacion_salir.lower() == 's':
                with open('notas.csv', 'a', newline='') as archivo_csv:
                    escritor_csv = csv.writer(archivo_csv)
                print('\n** HAS SALIDO DEL SISTEMA **')
                break
            else:
                print('\n** SALIDA CANCELADA, VOLVIENDO AL MENÚ PRINCIPAL **')
        else:
            print('\n** OPCIÓN NO VÁLIDA, INGRESE EL NÚMERO DE ALGUNA OPCIÓN MOSTRADA **')

def submenu_consultas():
    while True:
        print("\n---------------------------------------------")
        print("        SUBMENÚ CONSULTAS Y REPORTES")
        print("---------------------------------------------")
        print("1. Consulta por período")
        print("2. Consulta por folio")
        print("3. Consulta por cliente")
        print("4. Regresar al menú principal")

        subopcion = validacion_num("\nIngrese el número de la operación que desea realizar: ")

        if 1 <= subopcion <= 4:
            if subopcion == 1:
                consulta_periodo()
            elif subopcion == 2:
                consulta_folio()
            elif subopcion == 3:
                consulta_cliente()
            elif subopcion == 4:
                return
        else:
            print('\n*Opción no válida. Ingrese el número de alguna opción mostrada*')


menu_principal()



ModuleNotFoundError: ignored